In [1]:
import numpy as np

import mujoco

# import numpy as np
# import matplotlib.pyplot as plt

# from scipy.linalg import null_space
# from scipy.spatial.transform import Rotation as R

In [2]:
from main_2sept import LeapNodeMujoco, GraspClass
from inv_kin import GradientDescentIK, OnlyPosIK

model_path = '/home/sysidea/leap_hand_mujoco/model/leap hand/for_real.xml'
leap_hand = LeapNodeMujoco(model_path)

IK_index=GradientDescentIK('/home/sysidea/leap_hand_mujoco/model/leap hand/index_finger.xml')
IK_thumb=GradientDescentIK('/home/sysidea/leap_hand_mujoco/model/leap hand/thumb.xml')
IK_thumb_pos=OnlyPosIK('/home/sysidea/leap_hand_mujoco/model/leap hand/thumb.xml')
IK_middle=OnlyPosIK('/home/sysidea/leap_hand_mujoco/model/leap hand/middle.xml')
IK_tertiary=OnlyPosIK('/home/sysidea/leap_hand_mujoco/model/leap hand/tertiary.xml')

In [3]:
mujoco.mj_step(leap_hand.m,leap_hand.d)

pos_worldi1=leap_hand.d.site(leap_hand.m.site('contact_index1').id).xpos.reshape(3)
rot_worldi1=leap_hand.d.site(leap_hand.m.site('contact_index1').id).xmat.reshape(3,3)
pos_worldt1=leap_hand.d.site(leap_hand.m.site('contact_thumb1').id).xpos.reshape(3)
rot_worldt1=leap_hand.d.site(leap_hand.m.site('contact_thumb1').id).xmat.reshape(3,3)

pos_worldi2=leap_hand.d.site(leap_hand.m.site('contact_index2').id).xpos.reshape(3)
rot_worldi2=leap_hand.d.site(leap_hand.m.site('contact_index2').id).xmat.reshape(3,3)
pos_worldt2=leap_hand.d.site(leap_hand.m.site('contact_thumb2').id).xpos.reshape(3)
rot_worldt2=leap_hand.d.site(leap_hand.m.site('contact_thumb2').id).xmat.reshape(3,3)

pos_world_palm=leap_hand.d.body(leap_hand.m.body('palm').id).xpos.reshape(3)
rot_world_palm=np.reshape(leap_hand.d.body(leap_hand.m.body('palm').id).xmat,[3,3])

pos_palmi1=pos_worldi1-pos_world_palm
rot_palmi1=rot_world_palm.T @ rot_worldi1
pos_palmt1=pos_worldt1-pos_world_palm
rot_palmt1=rot_world_palm.T @ rot_worldt1

pos_palmi2=pos_worldi2-pos_world_palm
rot_palmi2=rot_world_palm.T @ rot_worldi2
pos_palmt2=pos_worldt1-pos_world_palm
rot_palmt2=rot_world_palm.T @ rot_worldt2

In [4]:
result_index1=IK_index.calculate(pos_palmi1, rot_palmi1,'contact_index')
result_thumb1=IK_thumb.calculate(pos_palmt1, rot_palmt1,'contact_thumb')
result_index2=IK_index.calculate(pos_palmi2, rot_palmi2,'contact_index')
result_thumb2=IK_thumb.calculate(pos_palmt2, rot_palmt2,'contact_thumb')

print("result_index1",result_index1)
print("result_thumb1",result_thumb1)
print("result_index2",result_index2)
print("result_thumb2",result_thumb2)

result_index1 [1.03637883e+00 1.84994063e-04 4.67016263e-01 6.80227245e-02]
result_thumb1 [ 1.57001343e+00 -1.89591547e-04  4.25434924e-01 -4.25023042e-01]
result_index2 [ 1.34970901e+00  2.07837393e-04  4.72382367e-01 -2.50659767e-01]
result_thumb2 [ 1.57001343e+00 -1.89591547e-04  4.25434924e-01 -4.25023042e-01]


In [5]:
def J(model,data,site_name):
        # model=mujoco.MjModel.from_xml_path(xml_path)
        # data = mujoco.MjData(model)
        mujoco.mj_forward(model, data)
        jacp = np.zeros((3, model.nv))  # translation jacobian
        jacr = np.zeros((3, model.nv)) 

        site_id=model.site(site_name).id
        mujoco.mj_jacSite(model, data, jacp, jacr, site_id)

        return np.vstack((jacp, jacr))

In [6]:
index_model_path = '/home/sysidea/leap_hand_mujoco/model/leap hand/index_finger.xml'

index_m = mujoco.MjModel.from_xml_path(index_model_path)
index_d = mujoco.MjData(index_m)

index_d.qpos=result_index1
mujoco.mj_forward(index_m, index_d)
index_J=J(index_m,index_d,'contact_index')
print(index_J)

[[-1.14115164e-01 -4.64969878e-04 -6.80118100e-02 -3.19937787e-02]
 [ 0.00000000e+00 -6.96632983e-02  7.61649532e-06  4.60976973e-06]
 [-1.87688604e-02  7.85599298e-04  7.58854679e-03  1.00198855e-02]
 [ 0.00000000e+00 -8.60565469e-01  9.42248460e-05  9.42248460e-05]
 [ 1.00000000e+00  0.00000000e+00  9.99999983e-01  9.99999983e-01]
 [ 0.00000000e+00 -5.09339841e-01 -1.59199502e-04 -1.59199502e-04]]


In [8]:
thumb_model_path = '/home/sysidea/leap_hand_mujoco/model/leap hand/thumb.xml'

thumb_m = mujoco.MjModel.from_xml_path(thumb_model_path)
thumb_d = mujoco.MjData(thumb_m)

thumb_d.qpos=result_thumb1
mujoco.mj_forward(thumb_m, thumb_d)
thumb_J=J(thumb_m,thumb_d,'contact_thumb')
print(thumb_J)

[[-4.42184769e-20 -2.94453991e-04  9.04419197e-02  4.79958762e-02]
 [-1.21742115e-01 -2.92524279e-02 -5.75463041e-06  1.25514587e-06]
 [ 1.99142316e-04 -2.29016929e-05  2.92523850e-02  1.00197735e-02]
 [-1.00000000e+00  0.00000000e+00  1.89591546e-04  1.89591546e-04]
 [ 0.00000000e+00  7.82898636e-04 -9.99999676e-01 -9.99999676e-01]
 [-2.22044605e-16 -9.99999694e-01 -7.82898622e-04 -7.82898622e-04]]
